# Heating Mesh Tally on CAD geometry made from Components

This constructs a reactor geometry from 3 Component objects each made from points.

The Component made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This section makes the 3d geometry for the entire reactor from a input parameters.

In [ ]:
import paramak

my_reactor = paramak.BallReactor(
    inner_bore_radial_thickness=50,
    inboard_tf_leg_radial_thickness=55,
    center_column_shield_radial_thickness=50,
    divertor_radial_thickness=50,
    inner_plasma_gap_radial_thickness=50,
    plasma_radial_thickness=100,
    outer_plasma_gap_radial_thickness=50,
    firstwall_radial_thickness=1,
    blanket_radial_thickness=100,
    blanket_rear_wall_radial_thickness=10,
    elongation=2,
    triangularity=0.55,
    number_of_tf_coils=16,
    pf_coil_case_thicknesses=[10, 10, 10, 10],
    pf_coil_radial_thicknesses=[20, 50, 50, 20],
    pf_coil_vertical_thicknesses=[20, 50, 50, 20],
    pf_coil_radial_position=[440, 550, 550, 440],
    pf_coil_vertical_position=[230, 100, -100, -230],
    rear_blanket_to_tf_gap=5,
    outboard_tf_coil_radial_thickness=30,
    outboard_tf_coil_poloidal_thickness=30,
    rotation_angle=90,
)

# TF and PF coils can be added with additional arguments.
# see the documentation for more details 
# https://paramak.readthedocs.io/en/main/paramak.parametric_reactors.html

my_reactor.show()

Exports the 3D geometry to a DAGMC neutronics geometry. The plasma is not included as not many neutron interactions occur in the low density plasma

In [ ]:
# creates a h5m file of the geometry with material tags automatically assigned
my_reactor.export_dagmc_h5m(filename="dagmc.h5m", min_mesh_size=5, max_mesh_size=20)

Names of material tags can be found with the command line tool

In [ ]:
!mbsize -ll dagmc.h5m | grep 'NAME = mat:'

The next section defines the materials. This can be done using openmc.Materials or in this case strings that look up materials from the neutronics material maker.

In [ ]:

import neutronics_material_maker as nmm
import openmc



# simplified material definitions have been used to keen this example minimal
mat_pf_coil_1 = nmm.Material.from_library(name='copper').openmc_material
mat_pf_coil_1.name='pf_coil_1'

mat_pf_coil_2 = nmm.Material.from_library(name='copper').openmc_material
mat_pf_coil_2.name='pf_coil_2'

mat_pf_coil_3 = nmm.Material.from_library(name='copper').openmc_material
mat_pf_coil_3.name='pf_coil_3'

mat_pf_coil_4 = nmm.Material.from_library(name='copper').openmc_material
mat_pf_coil_4.name='pf_coil_4'

mat_pf_coil_case_1 = openmc.Material(name="pf_coil_case_1")
mat_pf_coil_case_1.add_element("Fe", 1, "ao")
mat_pf_coil_case_1.set_density("g/cm3", 8.96)

mat_pf_coil_case_2 = openmc.Material(name="pf_coil_case_2")
mat_pf_coil_case_2.add_element("Fe", 1, "ao")
mat_pf_coil_case_2.set_density("g/cm3", 8.96)

mat_pf_coil_case_3 = openmc.Material(name="pf_coil_case_3")
mat_pf_coil_case_3.add_element("Fe", 1, "ao")
mat_pf_coil_case_3.set_density("g/cm3", 8.96)

mat_pf_coil_case_4 = openmc.Material(name="pf_coil_case_4")
mat_pf_coil_case_4.add_element("Fe", 1, "ao")
mat_pf_coil_case_4.set_density("g/cm3", 8.96)

mat_plasma = openmc.Material(name="plasma")
mat_plasma.add_element("H", 1, "ao")
mat_plasma.set_density("g/cm3", 0.00001)

mat_center_column_shield = openmc.Material(name="center_column_shield")
mat_center_column_shield.add_element("W", 1, "ao")
mat_center_column_shield.set_density("g/cm3", 19.3)

mat_outboard_firstwall = openmc.Material(name="firstwall")
mat_outboard_firstwall.add_element("Fe", 1, "ao")
mat_outboard_firstwall.set_density("g/cm3", 7.7)

mat_blanket = openmc.Material(name="blanket")
mat_blanket.add_elements_from_formula("Pb842Li158")
mat_blanket.set_density("g/cm3", 19.)

mat_divertor_upper = openmc.Material(name="divertor_upper")
mat_divertor_upper.add_element("W", 1, "ao")
mat_divertor_upper.set_density("g/cm3", 19.3)

mat_divertor_lower = openmc.Material(name="divertor_lower")
mat_divertor_lower.add_element("W", 1, "ao")
mat_divertor_lower.set_density("g/cm3", 19.3)

mat_supports = openmc.Material(name="supports")
mat_supports.add_element("Fe", 1, "ao")
mat_supports.set_density("g/cm3", 7.7)

mat_outboard_rear_blanket_wall = openmc.Material(name="outboard_rear_blanket_wall")
mat_outboard_rear_blanket_wall.add_element("Fe", 1, "ao")
mat_outboard_rear_blanket_wall.set_density("g/cm3", 7.7)

mat_inboard_tf_coils = nmm.Material.from_library(name='copper').openmc_material
mat_inboard_tf_coils.name = 'inboard_tf_coils'

mat_tf_coils = nmm.Material.from_library(name='copper').openmc_material
mat_tf_coils.name = 'tf_coil'

materials = openmc.Materials(
    [
        mat_pf_coil_1,
        mat_pf_coil_2,
        mat_pf_coil_3,
        mat_pf_coil_4,
        mat_pf_coil_case_1,
        mat_pf_coil_case_2,
        mat_pf_coil_case_3,
        mat_pf_coil_case_4,
        mat_plasma,
        mat_center_column_shield,
        mat_outboard_firstwall,
        mat_blanket,
        mat_divertor_upper,
        mat_divertor_lower,
        mat_supports,
        mat_outboard_rear_blanket_wall,
        mat_inboard_tf_coils,
        mat_tf_coils,
    ]
)

This next section builds the geometry, this is achieved using a filled CSG geometry with reflecting surfaces for the 90 degree wedge

In [ ]:

import math

# makes use of the dagmc geometry
dag_univ = openmc.DAGMCUniverse("dagmc.h5m")

# prints all the material tags used in the dagmc geometry
print(dag_univ.material_names)

# creates an edge of universe boundary surface
vac_surf = openmc.Sphere(r=10000, surface_id=9999, boundary_type="vacuum")

# adds reflective surface for the sector model at 0 degrees
reflective_1 = openmc.Plane(
    a=math.sin(0),
    b=-math.cos(0),
    c=0.0,
    d=0.0,
    surface_id=9991,
    boundary_type="reflective",
)

# adds reflective surface for the sector model at 90 degrees
reflective_2 = openmc.Plane(
    a=math.sin(math.radians(90)),
    b=-math.cos(math.radians(90)),
    c=0.0,
    d=0.0,
    surface_id=9990,
    boundary_type="reflective",
)

# specifies the region as below the universe boundary and inside the reflective surfaces
region = -vac_surf & -reflective_1 & +reflective_2

# creates a cell from the region and fills the cell with the dagmc geometry
containing_cell = openmc.Cell(cell_id=9999, region=region, fill=dag_univ)

my_geometry = openmc.Geometry(root=[containing_cell])


This next step makes a simple point source.

In [ ]:
# creates a simple isotropic neutron source in the center with 14MeV neutrons
my_source = openmc.IndependentSource()

# the distribution of radius is just a single value at the plasma major radius
radius = openmc.stats.Discrete([293.], [1])

# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])

# the distribution of source azimuthal angles values is a uniform distribution between 0 and 0.5 Pi
# these angles must be the same as the reflective angles
angle = openmc.stats.Uniform(a=0., b=math.radians(90))

# this makes the ring source using the three distributions and a radius
my_source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))

# sets the direction to isotropic
my_source.angle = openmc.stats.Isotropic()

# sets the energy distribution to a Muir distribution neutrons
my_source.energy = openmc.stats.muir(e0=14080000.0, m_rat=5.0, kt=20000.0)

In [ ]:
# specifies the simulation computational intensity
settings = openmc.Settings()
settings.batches = 10
settings.particles = 10000
settings.inactive = 0
settings.run_mode = "fixed source"
settings.source = my_source

Makes a 3D mesh tally

In [ ]:
# creates a mesh that covers the geometry
mesh = openmc.RegularMesh().from_domain(
    my_geometry, # the corners of the mesh are being set automatically to surround the geometry
    dimension=[100, 100, 100] # only 1 cell in the Y dimension
)

# makes a mesh tally using the previously created mesh and records heating on the mesh
mesh_tally = openmc.Tally(name="heating_on_mesh")
mesh_filter = openmc.MeshFilter(mesh)
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ["heating"]

heating_tally = openmc.Tally(name="heating")
heating_tally.scores = ["heating"]

# groups the two tallies
tallies = openmc.Tallies([mesh_tally, heating_tally])

starts the simulation

In [ ]:
# builds the openmc model
my_model = openmc.Model(
    materials=materials, geometry=my_geometry, settings=settings, tallies=tallies
)

# starts the simulation
statepoint_filename = my_model.run()

extracts the 3d mesh tally result and plots it as a vtk

In [ ]:

# open the results file
sp = openmc.StatePoint(statepoint_filename)

# access the tally using pandas dataframes
heating_tally = sp.get_tally(name="heating")

# print cell tally results with unit conversion
# raw tally result is multipled by 4 as this is a sector model of 1/4 of the total model (90 degrees from 360)
# raw tally result is divided by 1e6 to convert the standard units of eV to MeV
print(f"The heating of {4*heating_tally.mean.sum()/1e6} MeV per source particle is deposited")
print(f"Standard deviation on the heating tally is {heating_tally.std_dev.sum()}")

# extracts the mesh tally result
heating_mesh_tally = sp.get_tally(name="heating_on_mesh")

mesh.write_data_to_vtk(
    datasets={'heating_mean': heating_mesh_tally.mean},
    filename="reactor_heating_on_mesh.vtk",
)

The vtk file should have appeared in the file explorer to the left. Right mouse click on the file to download it and then open with Paraview